In [4]:
# Define the file path
file_path = './../Person-Tailored Gesture Classification/results/speed.txt'

# Read and parse the file
results = {}

with open(file_path, 'r') as f:
    for line in f:
        name, speed_str = line.split(':', 1)
        name = name.strip()
        speed = float(speed_str.strip())
        results[name] = speed

# Display the parsed results
speed_gpu = []
speed_cpu = []
for name, speed in results.items():
    if name.endswith('0'):
        speed_gpu.append(speed)
    else:
        speed_cpu.append(speed)
    print(f"Name: {name}, Average Speed: {speed:.2f}ms")

Name: Target_0_device_0, Average Speed: 18.25ms
Name: Target_1_device_0, Average Speed: 12.71ms
Name: Target_2_device_0, Average Speed: 14.85ms
Name: Target_3_device_0, Average Speed: 12.51ms
Name: Target_4_device_0, Average Speed: 13.04ms
Name: Target_5_device_0, Average Speed: 16.89ms
Name: Target_6_device_0, Average Speed: 12.87ms
Name: Target_0_device_cpu, Average Speed: 2495.70ms
Name: Target_1_device_cpu, Average Speed: 2579.06ms
Name: Target_2_device_cpu, Average Speed: 2664.09ms
Name: Target_3_device_cpu, Average Speed: 2727.76ms
Name: Target_4_device_cpu, Average Speed: 2718.65ms
Name: Target_5_device_cpu, Average Speed: 2459.30ms
Name: Target_6_device_cpu, Average Speed: 2698.42ms


In [6]:
sum(speed_gpu) / len(speed_gpu)

14.44419366972787

In [8]:
sum(speed_cpu) / len(speed_cpu)

2620.4244369552252

In [3]:
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
import pandas as pd

# Initialize lists to collect results
all_overall_results = []
all_class_metrics = []

for t in range(7):
    print(f'Participant No. {t} - evaluation on SIGGI_full')
    
    # Load ground truth and predictions JSON files
    ground_truth_file = f'./../datasets/SIGGI/full/{t}.json'
    predictions_file = f'./../Person-Tailored Gesture Classification/results/gpu/predictions_{t}.json' 
    
    # Load ground truth labels
    with open(ground_truth_file, 'r') as f:
        ground_truth = json.load(f)

    # Load predictions
    with open(predictions_file, 'r') as f:
        predictions = json.load(f)

    # Extract ground truth labels
    gt_labels = {item['image_id']: item['category_id'] for item in ground_truth['annotations']}

    # Extract predicted labels
    pred_labels = {item['image_id']: item['category_id'] for item in predictions['predictions']}  

    # Ensure the prediction keys match the ground truth keys
    image_ids = list(gt_labels.keys())
    y_true = [gt_labels[img_id] for img_id in image_ids]
    y_pred = [pred_labels.get(img_id, None) for img_id in image_ids]

    # Calculate metrics    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Create a DataFrame for overall results
    overall_results_df = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Average Score': [accuracy, precision, recall, f1]
    })
    
    # Append to list
    all_overall_results.append(overall_results_df)
    
    # Get unique class labels
    classes = sorted(set(y_true + y_pred))

    # Calculate metrics for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=classes, average=None)

    # Create a DataFrame for per-class metrics
    class_metrics_df = pd.DataFrame({
        'Class': classes,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })
    
    # Append to list
    all_class_metrics.append(class_metrics_df)
    
    # Print the DataFrames
    print()
    print(class_metrics_df.to_string(index=False))
    print()
    print(overall_results_df.to_string(index=False))

Participant No. 0 - evaluation on SIGGI_full

 Class  Precision   Recall  F1 Score
     0    1.00000 0.966667  0.983051
     1    0.84375 0.931034  0.885246
     2    0.75000 0.750000  0.750000
     3    0.75000 1.000000  0.857143
     4    1.00000 0.857143  0.923077
     5    1.00000 0.857143  0.923077

   Metric  Average Score
 Accuracy       0.933333
Precision       0.890625
   Recall       0.893664
 F1 Score       0.886932
Participant No. 1 - evaluation on SIGGI_full

 Class  Precision   Recall  F1 Score
     0   0.923077 1.000000  0.960000
     1   0.909091 0.869565  0.888889
     2   1.000000 0.933333  0.965517
     3   1.000000 0.888889  0.941176
     4   1.000000 0.600000  0.750000
     5   0.750000 0.750000  0.750000

   Metric  Average Score
 Accuracy       0.925000
Precision       0.930361
   Recall       0.840298
 F1 Score       0.875930
Participant No. 2 - evaluation on SIGGI_full

 Class  Precision   Recall  F1 Score
     0   0.909091 1.000000  0.952381
     1   0.925926 

In [8]:
# Aggregate overall results
aggregate_overall_results_df = pd.concat(all_overall_results).groupby('Metric').mean().reset_index()

# Aggregate per-class metrics
all_class_metrics_df = pd.concat(all_class_metrics).groupby('Class').mean().reset_index()

# Print aggregated results
print("Aggregated Per-Class Metrics:")
print(all_class_metrics_df.to_string(index=False))

print("\nAggregated Overall Results:")
print(aggregate_overall_results_df.to_string(index=False))

Aggregated Per-Class Metrics:
 Class  Precision   Recall  F1 Score
     0   0.953146 0.988015  0.969764
     1   0.856625 0.878839  0.864089
     2   0.858163 0.833163  0.842229
     3   0.935714 0.880042  0.897462
     4   0.906349 0.782160  0.834133
     5   0.907143 0.810544  0.854307

Aggregated Overall Results:
   Metric  Average Score
 Accuracy       0.918958
 F1 Score       0.876997
Precision       0.902857
   Recall       0.862127


In [ ]:
#extract cases where target present & correctly identified
# calc. average precision etc